Explores shapefiles and how they can be used.

In [1]:
import os
import time
import glob

import shapely
import pandas as pd
from tqdm import tqdm

import gmaps
import geopandas as gpd

In [2]:
key_path = "/Users/administrator/Documents/Projects/abq_crime/api_key.txt"

# API Authentication.
with open(key_path) as f:
    api_key = f.readline()
    f.close()

# Initialize a session.
gmaps.configure(api_key)

### ZipCodes Data

In [6]:
shapefile = gpd.read_file("/Users/administrator/Documents/Projects/abq_crime/shapefiles/zipcodes/zipcodes.shp")
print(shapefile)

    OBJECTID  ZIPCODE created_us  created_da last_edite  last_edi_1  \
0        145    87026       None        None       None        None   
1        141    87048       None        None       None        None   
2        143    87015       None        None       None        None   
3         31    87114       None        None       None        None   
4        132    87120       None        None       None        None   
5         22    87115       None        None       None        None   
6         65    87104       None        None       None        None   
7        144    87047       None        None       None        None   
8         28    87121       None        None       None        None   
9         29    87068       None        None       None        None   
10        41    87131       None        None       None        None   
11        82    87102       None        None       None        None   
12        47    87112       None        None       None        None   
13    

This data seems to be the most promising, so we are going to use this as the data.

### Plotting the Zip Codes

In [7]:
import pyproj

In [8]:
# Extract data from the shapefile.
shapefile = gpd.read_file("/Users/administrator/Documents/Projects/abq_crime/shapefiles/zipcodes/zipcodes.shp")

In [9]:
def epsg2258_to_epsg4326(coord):
    """Converts coordinates from EPSG:2258 to EPSG:4326.

    Parameters
    ----------
    coord : tuple (int, int)
        Tuple containing the coordinates in the EPSG:2258 standard.

    Returns
    -------
    trans_coords : tuple (int, int)
        Tuple containing the coordinates in the EPSG:4326 standard.

    """
    transformer = pyproj.Transformer.from_crs("epsg:2258", "epsg:4326")
    (x_proj, y_proj) = transformer.transform(coord[0], coord[1])

    return (x_proj, y_proj)

#### Example of a Zip Code with Two Polygons

In [14]:
# Shapefile index 23 contains two polygons. Let's try converting the indices and then plot it on Google Maps.
#coordinates = shapefile["geometry"][23][0].exterior.coords[:] + shapefile["geometry"][23][1].exterior.coords[:]
coordinates = shapefile["geometry"][23][0].exterior.coords[:]
for idx in tqdm(range(len(coordinates))):
    coordinates[idx] = epsg2258_to_epsg4326(coordinates[idx])

100%|██████████| 42/42 [00:05<00:00,  7.20it/s]


Now that the coordinates are in (latitude, longitude) format we can plot them using gmaps.

In [15]:
key_path = "/Users/administrator/Documents/Projects/abq_crime/api_key.txt"
with open(key_path) as f:
    api_key = f.readline()
    f.close()

gmaps.configure(api_key=api_key)

In [22]:
# Draw the Polygon using gmaps.
fig = gmaps.figure(center=(35.08541434188005, -106.65083179442777), zoom_level=13)
zipcodes_polygon = gmaps.Polygon(coordinates, stroke_color="red", fill_color=(217, 136, 128),)
drawing = gmaps.drawing_layer(features=[zipcodes_polygon], show_controls=False,)
fig.add_layer(drawing)
fig

Figure(layout=FigureLayout(height='420px'))

The result is unfortunately not pretty (since the two Polygons are connected by a line), so we should actually split them.

### Plotting all Zipcodes with Weights

In [1]:
import time

import shapely
import pyproj
import pandas as pd
from tqdm import tqdm

import gmaps
import geopandas as gpd

In [2]:
# Authenticate the API Key.
key_path = "/Users/administrator/Documents/Projects/abq_crime/api_key.txt"
with open(key_path) as f:
    api_key = f.readline()
    f.close()

# Initialize a session.
gmaps.configure(api_key=api_key)

In [3]:
# Load the Zip Code shape file.
shapefile_path = "/Users/administrator/Documents/Projects/abq_crime/shapefiles/zipcodes/zipcodes.shp"
zipcodes_shapefile = gpd.read_file(shapefile_path)

# Keep only the ZIPCODE and geometry columns.
keep_columns = ["ZIPCODE", "geometry"]
zipcodes_shapefile = zipcodes_shapefile[keep_columns]

In [4]:
# Utility functions.
def convert_coordinates(coordinates, transformer=None):
    """Converts coordinates from the EPSG:2258 standard to the EPSG:4326 standard.

    Parameters
    ----------
    coordinates : tuple (int, int)
        Tuple containing the coordinates in the EPSG:2258 standard.
    transformer : pyproj.transformer.Transformer instance
        Instance of a pyproj transformer used for converting the coordinates, by default None. If no transformer is specified, then the default transformer that converts EPSG:2258 to EPSG:4326 is used.
    
    Returns
    -------
    trans_coordinates : tuple (int, int)
        Tuple containing the coordinates in the EPSG:4326 standard.

    """
    if transformer is None:
        transformer = pyproj.Transformer.from_crs("epsg:2258", "epsg:4326")
    
    trans_coordinates = transformer.transform(coordinates[0], coordinates[1])

    return trans_coordinates

def batch_convert_coordinates(coord_list, transformer=None):
    """Converts a batch of coordinates from the EPSG:2258 standard to the EPSG:4326 standard.

    Parameters
    ----------
    coord_list : list of tuples (int, int)
        List containing tuples containing the coordinates in the EPSG:2258 standard.
    transformer : pyproj.transformer.Transformer instance
        Instance of a pyproj transformer used for converting the coordinates, by default None. If no transformer is specified, then the default transformer that converts EPSG:2258 to EPSG:4326 is used.

    Returns
    -------
    trans_coord_list : list of tuples (int, int)
        List containing tuples of coordinates in the EPSG:4326 standard.

    """
    if transformer is None:
        transformer = pyproj.Transformer.from_crs("epsg:2258", "epsg:4326")
    
    trans_coord_list = list()
    for coord in coord_list:
        trans_coord_list.append(convert_coordinates(coord, transformer=transformer))

    return trans_coord_list

In [5]:
# Create duplicate entries for zip codes that contain separated convex sets.
multipolygon_indices = [type(entry) == shapely.geometry.multipolygon.MultiPolygon for entry in zipcodes_shapefile["geometry"]]
multipolygon_indices = [idx for idx, x in enumerate(multipolygon_indices) if x == True]

In [ ]:
# For those indices above, create new entries at the end of the DataFrame, containing the ZIPCODE and geometry for each separate Polygon.
for indices in multipolygon_indices:
    
